# TwitterAPI - Get tweets

In [7]:
import tweepy
import json
import pandas as pd
import csv
import time
import os
from dotenv import load_dotenv
load_dotenv()

True

## Keys and tokens for the project

In [8]:
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
access_token = os.getenv('access_token')
access_token_secret = os.getenv('access_token_secret')

## Autenticacion

In [9]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Get Tweets

In [34]:
def getTweets(query_string, numTweets, numRuns):
    
    # We cannot make large API call in one go.
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['name', 'username', 'acctdesc', 'user_location',
     'tweet_full_text', 'tweet_place_name', 'tweet_place_type', 'tweet_place_bounding_box_type', 'tweet_place_bounding_box_coordinates', 'tweet_url'])
    program_start = time.time()
    for i in range(0, numRuns):
        start_run = time.time()
        tweets = tweepy.Cursor(api.search, q=query_string, tweet_mode='extended').items(numTweets)
        # Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
        # Begin scraping the tweets:
        noTweets = 0
        for tweet in tweet_list:
                start_url = 'https://twitter.com/twitter/statuses/'
                if not tweet.full_text.startswith('RT'):
                        # Pull the values
                        tweet_id = tweet.id
                        name = tweet.user.name
                        username = tweet.user.screen_name
                        acctdesc = tweet.user.description
                        user_location = tweet.user.location
                        tweet_full_text = tweet.full_text
                        tweet_place_bounding_box_type = tweet.place.bounding_box.type
                        tweet_place_bounding_box_coordinates = tweet.place.bounding_box.coordinates
                        tweet_url = start_url + str(tweet_id)
                        try:
                                tweet_place_name =  tweet.place.name
                                tweet_place_type = tweet.place.type
                        except AttributeError:
                                tweet_place_name =  'None'
                                tweet_place_type = 'None'
        # Data for the dataset:
                        ith_tweet = [name, username, acctdesc, user_location, tweet_full_text, tweet_place_name, tweet_place_type, tweet_place_bounding_box_type, tweet_place_bounding_box_coordinates, tweet_url]

        # Append to db_tweets
                        db_tweets.loc[len(db_tweets)] = ith_tweet
        # increase counter - noTweets  
                        noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('Número de tweets obtenidos en la corrida {} es {}'.format(i + 1, noTweets))
        print('tiempo que se tomo la corrida {} para completar la tarea es {} mins'.format(i+1, duration_run))
        
    time.sleep(920) #15 minute sleep time
    # Store dataframe in csv
    db_tweets.to_csv('test.csv', index = False)
    
    program_end = time.time()
    print('El proceso se ha completado!')
    print('Total time: {} minutes.'.format(round(program_end - program_start)/60, 2))

In [35]:
# Initialise these variables:
places = api.geo_search(query="EC", granularity="country")
place_id = places[0].id
query_string = "#PlanVacunación9100 OR #VacunaCOVID19 OR #JuntosLoLogramos place:"+place_id
numTweets = 5
numRuns = 2
# Call the function scraptweets
getTweets(query_string, numTweets, numRuns)

Número de tweets obtenidos en la corrida 1 es 5
tiempo que se tomo la corrida 1 para completar la tarea es 0.01 mins
Número de tweets obtenidos en la corrida 2 es 5
tiempo que se tomo la corrida 2 para completar la tarea es 0.01 mins
El proceso se ha completado!
Total time: 0.03333333333333333 minutes.
